In [57]:
import pandas as pd
from itertools import zip_longest

def split_clean(s) : 
    return [t.strip() for t in str(s).split(";") if t.strip() != ""]

file = "DataEntryGridExport_sept.-03-2025-11-02-20.xls"
df = pd.read_excel(file, header=[0, 1, 2])

# Concaténer les 3 niveaux en une seule chaîne lisible
df.columns = [
    "_".join([str(x).strip() 
              for x in tup 
              if not str(x).startswith("Unnamed") and str(x).strip() not in ("nan", "None", "")])
    for tup in df.columns.to_flat_index()
]
print(df.columns)
df.to_csv("output.csv")
df2 = df[["Code labo", "Instance #","PRE-PCR-MON (0,1)_Amorces (Standard, Rep [replicateid])","PRE-PCR-MON (0,1)_Programme PCR (Standard, Rep [replicateid])"]]
df2 = df2.rename(columns={"PRE-PCR-MON (0,1)_Amorces (Standard, Rep [replicateid])" : "Amorces", "PRE-PCR-MON (0,1)_Programme PCR (Standard, Rep [replicateid])" : "ProgrammePCR","Instance #" : "Instance"})
df2["Amorces_liste"]= df2["Amorces"].apply(split_clean)
df2["Programme_liste"] = df2["ProgrammePCR"].apply(split_clean)
df2["pairs"] = df2.apply(lambda r : list(zip_longest(r["Amorces_liste"], r["Programme_liste"], fillvalue = pd.NA)), axis=1)
macron = df2.explode("pairs", ignore_index=True)
macron[["Amorces","ProgrammePCR"]] = macron["pairs"].apply(pd.Series)
result = macron[["Code labo", "Instance", "Amorces", "ProgrammePCR"]].copy()


Index(['Echantillon', 'Code labo', 'Tiers demandeur', 'Produit', 'Instance #',
       'Rep', 'PRE-PCR-MON (0,1)_Operateur (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Date analyse (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Facteur de dilution (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Enzyme utilise (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Groupe Amorce (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Amorces (Standard, Rep [replicateid])',
       'PRE-PCR-MON (0,1)_Programme PCR (Standard, Rep [replicateid])',
       'EXT-ADN-MON (0,1)_Operateur (Standard, Rep [replicateid])',
       'EXT-ADN-MON (0,1)_Date analyse (Standard, Rep [replicateid])',
       'PCR-MON (0,1)_Operateur (Standard, Rep [replicateid])',
       'PCR-MON (0,1)_Date analyse (Standard, Rep [replicateid])',
       'AGAROSE-MON (0,1)_Operateur (Standard, Rep [replicateid])',
       'AGAROSE-MON (0,1)_Date analyse (Standard, Rep [replicateid])',
     

,Code labo,Instance,Amorces,ProgrammePCR
0,BAR250681,1,16sMAV,55 35
1,BAR250681,1,Bar12sV5,55 35
2,BAR250681,1,Bar16svSarri,55 35
3,BAR250681,1,F_mlCOIint_R_jgHCO2198,62 46 TD
4,BAR250681,1,ZBJ,Prog. ZBJ insectes
...,...,...,...,...
180,BAR250687,1,ALITS_12_FR,55 35
181,BAR250687,1,TufA,55 35
182,BAR250688,1,TrnLgh,55 35
183,BAR250688,1,ITS1_F_rc_ITS4_Rev2,55 35


In [83]:
import pandas as pd
import numpy as np
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter

# ========= 1) Préparation des données =========


def sort_within_program(group: pd.DataFrame) -> pd.DataFrame:
    counts = group["Amorces"].value_counts()
    group = group.copy()
    group["Amorce_count"] = group["Amorces"].map(counts)
    return group.sort_values(
        ["Amorce_count", "Amorces", "Code labo", "Instance_num"],
        ascending=[False, True, True, True]
    ).drop(columns="Amorce_count")

df_sorted = (df.groupby("ProgrammePCR", group_keys=False)
               .apply(sort_within_program)
               .reset_index(drop=True))

def assign_plates_columns(group: pd.DataFrame, plate_size: int = 96) -> pd.DataFrame:
    n = len(group)
    idx = np.arange(n)

    plate = idx // plate_size + 1
    pos   = idx % plate_size

    # remplissage par colonnes (A01→H01→A02→H02…)
    col_num = pos // 8 + 1
    row_num = pos % 8

    rows = np.array(list("ABCDEFGH"))
    row_lbl = rows[row_num]
    well = [r + str(c).zfill(2) for r, c in zip(row_lbl, col_num)]

    out = group.copy()
    out["PlateNbr"] = plate
    out["Row"] = row_lbl
    out["Col"] = col_num
    out["Well"] = well
    return out

plates = (df_sorted.groupby("ProgrammePCR", group_keys=False)
          .apply(assign_plates_columns, plate_size=96)
          .reset_index(drop=True))

plates["Content"] = (
    plates["Code labo"].astype(str) + "_" +
    plates["Instance"].astype(str)  + "_" +
    plates["Amorces"].astype(str)
)

# ========= 2) Export vers un template =========
template_file = "MON-ENR-053_PCR NGS_V0_250523 - Copie.xlsx"
out_xlsx = "Template_rempli.xlsx"

wb = load_workbook(template_file)
ws = wb["Feuil1"]  # adapter si besoin

# Position d'ancrage dans le template
start_row = 5
start_col = 6

thin = Side(border_style="thin", color="000000")
border = Border(top=thin, bottom=thin, left=thin, right=thin)

current_row = start_row

# ordre des lignes (A..H) garanti
row_order = {ch: i for i, ch in enumerate("ABCDEFGH")}

for (prog, plate_nbr), sub_plate in plates.groupby(["ProgrammePCR", "PlateNbr"], sort=True):
    # lignes/colonnes réellement utilisées
    rows_used = sorted(sub_plate["Row"].unique(), key=row_order.get)
    cols_used = sorted(map(int, sub_plate["Col"].unique()))  # numérique 1..12
    nb_cols = len(cols_used) + 1  # +1 pour la colonne 'Row'

    # === Titre fusionné (largeur utile) ===
    title = f"{prog} — Plaque {plate_nbr}"
    ws.merge_cells(start_row=current_row, start_column=start_col,
                   end_row=current_row, end_column=start_col + nb_cols - 1)
    ctitle = ws.cell(row=current_row, column=start_col, value=title)
    ctitle.font = Font(bold=True)
    ctitle.alignment = Alignment(horizontal="center")
    current_row += 1

    # === En-têtes colonnes ===
    headers = ["", *cols_used]
    for j, val in enumerate(headers):
        cell = ws.cell(row=current_row, column=start_col + j, value=val)
        cell.border = border
        cell.font = Font(bold=True)
        cell.alignment = Alignment(horizontal="center")
    current_row += 1

    # === Matrice restreinte aux lignes/colonnes utiles ===
    mat = (sub_plate.pivot(index="Row", columns="Col", values="Content")
                    .reindex(index=rows_used, columns=cols_used))

    # On insère la première colonne 'Row' (A..H)
    block = mat.copy()
    block.insert(0, "Row", block.index)
    block = block.reset_index(drop=True)

    # Écriture explicite cellule par cellule à partir de (current_row, start_col)
    start_r = current_row
    nrows, ncols = block.shape  # inclut la colonne 'Row'
    for i in range(nrows):
        for j in range(ncols):
            val = block.iat[i, j]
            ws.cell(row=start_r + i,
                    column=start_col + j,
                    value=None if pd.isna(val) else val)

    end_r = start_r + nrows - 1
    end_c = start_col + ncols - 1

    # Bordures + centrage sur toute la zone écrite
    for r in ws.iter_rows(min_row=start_r, max_row=end_r,
                          min_col=start_col, max_col=end_c):
        for cell in r:
            cell.border = border
            cell.alignment = Alignment(horizontal="center")

    # Autofit colonnes (sans rétrécir si déjà plus large)
    for col in range(start_col, end_c + 1):
        max_len = 0
        for row in ws.iter_rows(min_row=start_r, max_row=end_r, min_col=col, max_col=col):
            for cell in row:
                if cell.value is not None:
                    max_len = max(max_len, len(str(cell.value)))
        col_letter = get_column_letter(col)
        target = 6 if col == start_col else max(max_len + 2, 12)
        current_w = ws.column_dimensions[col_letter].width or 0
        ws.column_dimensions[col_letter].width = max(current_w, target)

    # Ligne vide de séparation entre plaques
    current_row = end_r + 1 + 2

wb.save(out_xlsx)
print(f"Écrit : {out_xlsx}")


Écrit : Template_rempli.xlsx


/tmp/ipykernel_3818470/3867858463.py:21: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(sort_within_program)
/tmp/ipykernel_3818470/3867858463.py:47: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(assign_plates_columns, plate_size=96)


Écrit : Plaques96_empilees.xlsx
